In [1]:
import pandas as pd
from pathlib import Path
_
import PC_Fns
import hs
import filecmp
import numpy as np 
import imageio
import tarfile
import shutil
from PC_Fns import PandUPF
import decodeRAW_HK

In [39]:
wkdir = Path(r"C:\Users\ucasbwh\Desktop\20190925_101155_HRC_-20C_ABSR_MID")
files_sci = PC_Fns.Find_Files(wkdir, "RMAP_Sci*.txt", Recursive=False)

# Check for first file being empty due to labview bug
if files_sci[0].stat().st_size == 0:
    files_sci[0].unlink()
    del files_sci[0]
print(files_sci[6].stat().st_size)

# Find all last LDT parts
small_chunks = [files_sci.index(chunk) for chunk in files_sci if chunk.stat().st_size == 762030]


# Look for consecutive chunks and replace with .ignore
print(small_chunks)
repeat_chunks = [small_chunks[i+1] for i, x in enumerate(np.diff(small_chunks)==1) if x]
print(repeat_chunks)

wrong_files = [files_sci[i] for i in repeat_chunks]
print(wrong_files)

for wrong_item in wrong_files:
    #wrong_item.rename(wrong_item.with_suffix('.txt.ignore'))
    files_sci.remove(wrong_item)
print(files_sci)

762030
[6, 13, 20, 27, 34, 41, 42, 49, 56, 57, 64, 71, 78, 85, 86, 93, 100, 107, 114, 121, 128, 135, 142, 149]
[42, 57, 86]
[WindowsPath('C:/Users/ucasbwh/Desktop/20190925_101155_HRC_-20C_ABSR_MID/RMAP_Sci_20190925_101155_043.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/20190925_101155_HRC_-20C_ABSR_MID/RMAP_Sci_20190925_101155_058.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/20190925_101155_HRC_-20C_ABSR_MID/RMAP_Sci_20190925_101155_087.txt')]
[WindowsPath('C:/Users/ucasbwh/Desktop/20190925_101155_HRC_-20C_ABSR_MID/RMAP_Sci_20190925_101155_001.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/20190925_101155_HRC_-20C_ABSR_MID/RMAP_Sci_20190925_101155_002.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/20190925_101155_HRC_-20C_ABSR_MID/RMAP_Sci_20190925_101155_003.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/20190925_101155_HRC_-20C_ABSR_MID/RMAP_Sci_20190925_101155_004.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/20190925_101155_HRC_-20C_ABSR_MID/RMAP_Sci_20190925_101155_005.txt'), WindowsPat

In [46]:
proc_dir = Path(r"C:\WDir\PanCam EGSE\Logs\FS Calibration Reprocessed\20191011_095418_WAC_R5_-50c_ABSR\PROC")
unproc = "Unproc_HKTM.pickle"

RTM = pd.read_pickle(proc_dir / unproc)
Bin = RTM['RAW'].apply(lambda x: bytearray.fromhex(x))

TM = pd.DataFrame()
TM['Stat_PIU_En'] = PandUPF(Bin, 'u8', 42, 0)
TM['Stat_PIU_Pw'] = PandUPF(Bin, 'u8', 43, 0)

WACBin, HRCBin = decodeRAW_HK.Determ_CamRes(TM, Bin)
WACBin.head()

127     [53, 3, 0, 15, 12, 90, 2, 4, 0, 0, 61, 0, 7, 2...
1306    [53, 3, 0, 15, 16, 250, 2, 4, 0, 0, 61, 0, 7, ...
1308    [53, 3, 0, 15, 16, 252, 2, 4, 0, 0, 61, 0, 7, ...
1309    [53, 3, 0, 15, 16, 253, 2, 4, 0, 0, 61, 0, 7, ...
1310    [53, 3, 0, 15, 16, 254, 2, 4, 0, 0, 61, 0, 7, ...
Name: RAW, dtype: object

In [47]:
TM['WAC_CID'] = PandUPF(WACBin, 'u2', 44, 0)
TM[TM.WAC_CID == 3].head()
TM[44:].head(60)

Stat_PIU_En  Stat_PIU_Pw  WAC_CID
44             2            2      NaN
45             2            2      NaN
46             2            2      NaN
47             2            2      NaN
48             2            2      NaN
49             2            2      NaN
50             2            2      NaN
51             2            2      NaN
52             2            2      NaN
53             2            2      NaN
54             2            2      NaN
55             2            2      NaN
56             2            2      NaN
57             2            2      NaN
58             2            2      NaN
59             2            2      NaN
60             2            2      NaN
61             2            2      NaN
62             2            2      NaN
63             2            2      NaN
64             2            2      NaN
65             2            2      NaN
66             2            2      NaN
67             2            2      NaN
68             2            2      NaN
69             2            2      NaN
70             2            2      NaN
71             2            2      NaN
72             2            2      NaN
73             2            2      NaN
74             2            2      NaN
75             2            2      NaN
76             2            2      NaN
77             2            2      NaN
78             2            2      NaN
79             2            2      NaN
80             2            2      NaN
81             2            2      NaN
82             2            2      NaN
83             2            2      NaN
84             2            2      NaN
85             2            2      NaN
86             2            2      NaN
87             2            2      NaN
88             2            2      NaN
89             2            2      NaN
90             2            2      NaN
91             2            2      NaN
92             2            2      NaN
93             2            2      NaN
94             2            2      NaN
95             2            2      NaN
96             2            2      NaN
97             2            2      NaN
98             2            2      NaN
99             2            2      NaN
100            2            2      NaN
101            2            2      NaN
102            2            2      NaN
103            2            2      NaN

In [53]:
CamResSeries = Bin.apply(lambda x: x[44: 64])
repaired = Bin[127:1306].apply(lambda x: x[:44] + x[44: 64].replace(b'\xFF', b'\x00') + x[64:])
Bin[127:1306] = repaired


In [16]:
lv_dir = Path(r"C:\Users\ucasbwh\Desktop\LabView Dev")
files_hs = PC_Fns.Find_Files(lv_dir, "RMAP_H&S*.txt")
files_hk = PC_Fns.Find_Files(lv_dir, "RMAP_HK*.txt")

proc_dir = lv_dir / "PROC"
#proc_dir.mkdir()

print(files_hs)
print(files_hk)

[WindowsPath('C:/Users/ucasbwh/Desktop/LabView Dev/RMAP_H&S_20200204_154341_000.txt')]
[WindowsPath('C:/Users/ucasbwh/Desktop/LabView Dev/RMAP_HK_20200204_154341_000.txt')]


In [17]:
for curfile in files_hs:
    cur_name = curfile.stem
    hs_head = ['Time', 'RAW']
    hs_df = pd.read_csv(curfile, sep=r' \t ', header=None, names=hs_head)
    hs_df['RAW'] = hs_df['RAW'].apply(lambda x: x.replace('\t', ' '))
hs_df.to_pickle(proc_dir / "hs_raw.pickle")
hs_df.head(20)

Time  \
0   2020-02-04\t15:44:45.975   
1   2020-02-04\t15:44:46.011   
2   2020-02-04\t15:44:46.211   
3   2020-02-04\t15:44:46.411   
4   2020-02-04\t15:44:46.611   
5   2020-02-04\t15:44:46.811   
6   2020-02-04\t15:44:47.011   
7   2020-02-04\t15:44:47.211   
8   2020-02-04\t15:44:47.411   
9   2020-02-04\t15:44:47.611   
10  2020-02-04\t15:44:47.811   
11  2020-02-04\t15:44:48.011   
12  2020-02-04\t15:44:48.211   
13  2020-02-04\t15:44:48.411   
14  2020-02-04\t15:44:48.611   
15  2020-02-04\t15:44:48.811   
16  2020-02-04\t15:44:49.011   
17  2020-02-04\t15:44:49.211   
18  2020-02-04\t15:44:49.411   
19  2020-02-04\t15:44:49.611   

                                                  RAW  
0   80 00 00 00 00 58 00 00 C0 00 00 00 00 00 00 0...  
1   80 00 00 00 00 58 00 00 C0 00 00 00 00 00 00 0...  
2   80 00 00 00 00 58 00 00 C0 00 00 00 00 00 00 0...  
3   80 00 00 00 00 58 00 00 C0 00 00 00 00 00 00 0...  
4   80 00 00 00 00 58 00 00 C0 00 00 00 00 00 00 0...  
5   80 00 00 00 00 58 00 00 C0 00 00 00 00 00 00 0...  
6   80 00 00 00 00 58 00 01 C0 00 00 00 00 00 00 0...  
7   80 00 00 00 00 58 00 01 C0 00 00 00 00 00 00 0...  
8   80 00 00 00 00 58 00 01 C0 00 00 00 00 00 00 0...  
9   80 00 00 00 00 58 00 01 C0 00 00 00 00 00 00 0...  
10  80 00 00 00 00 58 00 01 C0 00 00 00 00 00 00 0...  
11  80 00 00 00 00 58 00 02 C0 00 00 00 00 00 00 0...  
12  80 00 00 00 00 58 00 02 C0 00 00 00 00 00 00 0...  
13  80 00 00 00 00 58 00 02 C0 00 00 00 00 00 00 0...  
14  80 00 00 00 00 58 00 02 C0 00 00 00 00 00 00 0...  
15  80 00 00 00 00 58 00 02 C0 00 00 00 00 00 00 0...  
16  80 00 00 00 00 58 00 03 C0 00 00 00 00 00 00 0...  
17  80 00 00 00 00 58 00 03 C0 00 00 00 00 00 00 0...  
18  80 00 00 00 00 58 00 03 C0 00 00 00 00 00 00 0...  
19  80 00 00 00 00 58 00 03 C0 00 00 00 00 00 00 0...

In [18]:
# Compare output
swis_dir = Path(r"C:\Users\ucasbwh\Desktop\NSVF Dev\PROC")
swis_hs = "hs_raw.pickle"

swis = pd.read_pickle(swis_dir / swis_hs)
swis.head(20)


Time                                                RAW
0   [296:058:009:386]   21 01 0C 00 41 00 3A 00 00 00 20 69 80 00 00 ...
1   [296:258:009:386]   21 01 0C 00 41 00 40 00 00 00 20 7C 80 00 00 ...
2   [296:456:009:386]   21 01 0C 00 41 00 45 00 00 00 20 B3 80 00 00 ...
3   [296:656:009:386]   21 01 0C 00 41 00 4A 00 00 00 20 23 80 00 00 ...
4   [296:856:009:386]   21 01 0C 00 41 00 4F 00 00 00 20 EC 80 00 00 ...
5   [297:056:009:386]   21 01 0C 00 41 00 54 00 00 00 20 C2 80 00 00 ...
6   [297:256:009:386]   21 01 0C 00 41 00 5B 00 00 00 20 52 80 00 00 ...
7   [297:456:009:386]   21 01 0C 00 41 00 60 00 00 00 20 8D 80 00 00 ...
8   [297:656:009:386]   21 01 0C 00 41 00 65 00 00 00 20 42 80 00 00 ...
9   [297:856:009:386]   21 01 0C 00 41 00 6A 00 00 00 20 D2 80 00 00 ...
10  [298:056:009:386]   21 01 0C 00 41 00 6F 00 00 00 20 1D 80 00 00 ...
11  [298:254:009:386]   21 01 0C 00 41 00 75 00 00 00 20 DA 80 00 00 ...
12  [298:456:009:386]   21 01 0C 00 41 00 7A 00 00 00 20 4A 80 00 00 ...
13  [298:656:009:386]   21 01 0C 00 41 00 7F 00 00 00 20 85 80 00 00 ...
14  [298:854:009:386]   21 01 0C 00 41 00 84 00 00 00 20 3F 80 00 00 ...
15  [299:054:009:386]   21 01 0C 00 41 00 89 00 00 00 20 BC 80 00 00 ...
16  [299:254:009:386]   21 01 0C 00 41 00 8F 00 00 00 20 89 80 00 00 ...
17  [299:454:009:386]   21 01 0C 00 41 00 94 00 00 00 20 A7 80 00 00 ...
18  [299:654:009:386]   21 01 0C 00 41 00 99 00 00 00 20 24 80 00 00 ...
19  [299:854:009:386]   21 01 0C 00 41 00 9E 00 00 00 20 F8 80 00 00 ...

In [4]:
hs.decode(proc_dir)

SpW Offset: %d 0


In [6]:
raw = hs_df['RAW'].apply(lambda x: bytearray.fromhex(x))
raw.head()

0    [128, 0, 0, 0, 0, 88, 0, 0, 192, 0, 0, 0, 0, 0...
1    [128, 0, 0, 0, 0, 88, 0, 0, 192, 0, 0, 0, 0, 0...
2    [128, 0, 0, 0, 0, 88, 0, 0, 192, 0, 0, 0, 0, 0...
3    [128, 0, 0, 0, 0, 88, 0, 0, 192, 0, 0, 0, 0, 0...
4    [128, 0, 0, 0, 0, 88, 0, 0, 192, 0, 0, 0, 0, 0...
Name: RAW, dtype: object

In [34]:
swis = swis['RAW'].apply(lambda x: bytearray.fromhex(x))
swis.head()

0    [33, 1, 12, 0, 65, 0, 58, 0, 0, 0, 32, 105, 12...
1    [33, 1, 12, 0, 65, 0, 64, 0, 0, 0, 32, 124, 12...
2    [33, 1, 12, 0, 65, 0, 69, 0, 0, 0, 32, 179, 12...
3    [33, 1, 12, 0, 65, 0, 74, 0, 0, 0, 32, 35, 128...
4    [33, 1, 12, 0, 65, 0, 79, 0, 0, 0, 32, 236, 12...
Name: RAW, dtype: object

In [13]:
fftest = pd.read_pickle(proc_dir / "hs.pickle")
test['Sci_Len'].value_counts()

2097200    4381
0          1673
Name: Sci_Len, dtype: int64

In [ ]:
f

In [5]:
hs.verify(proc_dir)aa

In [14]:
hk_head = ['Time', 'RAW']
for curfile in files_hk:
    dl = pd.read_csv(curfile, sep=r' \t ', header=None, names=hk_head, engine='python')
    dl['RAW'] = dl['RAW'].apply(lambda x: x.replace('\t', ' '))
    dl['Source'] = 'LabView'

dl.head()
dl.to_pickle(proc_dir / "Unproc_HKTM.pickle")

In [20]:
# Create RMAP_Sci versions and compare
files_sci = PC_Fns.Find_Files(lv_dir, "RMAP_Sci*.txt")
print(len(files_sci))
print(files_sci[0:2aa])

50
[WindowsPath('C:/Users/ucasbwh/Desktop/LabView Dev/RMAP_Sci_20200204_154341_000.txt'), WindowsPath('C:/Users/ucasbwh/Desktop/LabView Dev/RMAP_Sci_20200204_154341_001.txt')]


In [65]:
sci_head = ['Time', 'RAW']
dl = pd.read_csv(files_sci[1], sep=r' \t ', header=None, names=sci_head)
dl['RAW'] = dl['RAW'].apply(lambda x: x.replace('\t', ' '))
raw = dl['RAW'].apply(lambda x: bytearray.fromhex(x))
write_dir = proc_dir / "IMG_SPW"
write_dir.mkdir()
write_file = write_dir / "test.pci_raw"
wf = open(write_file, 'w+b')
wf.write(raw[0])
wf.close()


In [66]:
write_file = write_dir / '000.pci_spw'
wf = open(write_file, 'w+b')

for curfile in files_sci:
    dl = pd.read_csv(curfile, sep=r' \t ', header=None, names=sci_head)
    if dl.shape[0] == 0:
        next
    elif dl.shape[0] > 1:
        print("Warning two lines in single file")
    else:
        dl['RAW'] = dl['RAW'].apply(lambda x: x.replace('\t', ' '))
        raw = dl['RAW'].apply(lambda x: bytearray.fromhex(x))
        if write_file.stat().st_size >= 2097200:
            wf.close()
            cur_ldt = curfile.stem.split('_')[-1]
            write_file = write_dir / (cur_ldt + ".pci_spw")
            wf = open(write_file, 'w+b')
        wf.write(raw[0])

wf.close()
        

In [109]:
from shutil import copyfile

bin_files = PC_Fns.Find_Files(lv_dir, '*.bin')
spw_files = PC_Fns.Find_Files(lv_dir, '*.pci_spw')
img_dir = proc_dir / "IMG_RAW"
#img_dir.mkdir()
unmatched = [];

for curfile in bin_files:
    matched = False

    for ref in spw_files:
        if filecmp.cmp(curfile, ref, shallow=False):
            print("Files match", curfile.stem, ref.name)
            spw_files.remove(ref)
            #ref.unlink()
            copyfile(curfile, img_dir / (curfile.stem + ".pci_raw"))
            matched = True
            break
            
        else:
            print("No match   ", curfile.stem, ref.name)
            
    if not matched: print("Nothing here mate")


Files match 200204_155027_FS-L01_+163_601ms_PH-0.00000 000.pci_spw
Files match 200204_155216_FS-R01_+149_488ms_PH-0.00000 008.pci_spw
No match    200204_160316_FS_HRC_+0_0.0000ms_PH-0.00000 015.pci_spw
No match    200204_160316_FS_HRC_+0_0.0000ms_PH-0.00000 022.pci_spw
No match    200204_160316_FS_HRC_+0_0.0000ms_PH-0.00000 029.pci_spw
No match    200204_160316_FS_HRC_+0_0.0000ms_PH-0.00000 036.pci_spw
Files match 200204_160316_FS_HRC_+0_0.0000ms_PH-0.00000 043.pci_spw


In [32]:
lv_dir = Path(r"C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal")
spw_files = PC_Fns.Find_Files(lv_dir, 'RMAP_Sci*.txt')
[print(item ,item.stat().st_size) for item in spw_files[7::7]]

C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal\ARCHIVE\RMAP_Sci\RMAP_Sci_20200205_135646_007.txt 762030
C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal\ARCHIVE\RMAP_Sci\RMAP_Sci_20200205_135646_014.txt 762030
C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal\ARCHIVE\RMAP_Sci\RMAP_Sci_20200205_135646_021.txt 762030
C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal\ARCHIVE\RMAP_Sci\RMAP_Sci_20200205_135646_028.txt 762030
C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal\ARCHIVE\RMAP_Sci\RMAP_Sci_20200205_135646_035.txt 762030
C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal\ARCHIVE\RMAP_Sci\RMAP_Sci_20200205_135646_042.txt 762030
C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal\ARCHIVE\RMAP_Sci\RMAP_Sci_20200205_135646_049.txt 762030
C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal\ARCHIVE\RMAP_Sci\RMAP_Sci_20200205_135646_056.txt 762030
C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal\ARCHIVE\RMAP_Sci\RMAP_Sci_20200205_135646_063.txt 762030
C:\Users\ucasbwh\Desktop\20200205_135646_GeomC

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [29]:
filecmp.cmp(spw_files[98], spw_files[99], shallow=False)

False

In [30]:
d1 = pd.read_csv(spw_files[98], sep=r' \t ', header=None, names=['Time', 'RAW'])
d2 = pd.read_csv(spw_files[99], sep=r' \t ', header=None, names=['Time', 'RAW'])

d1.RAW == d2.RAW

0    True
Name: RAW, dtype: bool

In [34]:
spw_files[0].stat().st_size

0

In [38]:
del spw_files[0]
len(spw_files)

1239

In [6]:
lv_dir = Path(r"C:\Users\ucasbwh\Desktop\20200205_135646_GeomCal")
bin_files = PC_Fns.Find_Files(lv_dir, "*.bin", Recursive=False)
spw_files = PC_Fns.Find_Files(lv_dir / "PROC", "*.pci_spw")
print(len(bin_files))
print(len(spw_files))

28
34


In [11]:
bin_len = bin_files[0].stat().st_size
tmp_file = lv_dir / "PROC" / "tmp_cmp.bin"
tmp = open(tmp_file, 'wb')

#for curfile in spw_files:
ref = spw_files[0]
with open(ref, 'rb') as sf:
    tmp.write(sf.read(bin_len))
    print(filecmp.cmp(bin_files[0], tmp_file, shallow=False))
    spw_files.remove(ref)

    pci_raw_file = img_dir / (bin_files[0].stem + "_repaired" + ref.stem + ".pci_raw")
    copyfile(ref, pci_raw_file)
    #create_repairedjson(pci_raw_file)

True


In [12]:
tmp.close()

In [15]:
wkdir = Path(r"C:\Users\ucasbwh\Desktop\ImageCompare")
raw_file = PC_Fns.Find_Files(wkdir, "*_repaired*.pci_raw")
print(raw_file)

[WindowsPath('C:/Users/ucasbwh/Desktop/ImageCompare/200205_102357_FS-L01_+153_501ms_repaired162.pci_raw')]


In [21]:
with open(raw_file[0], 'rb') as f:
    img_rawheader = f.read(48)
    raw_data = np.fromfile(f, dtype='>u2')

    img = raw_data.reshape(1024,1024)
    Br_img = np.rot90(img, k=3)

    write_filename = "test.png"
    write_file = wkdir / write_filename

    if write_file.exists():
        write_file.unlink()
    
    imageio.imwrite(write_file, Br_img)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ucasbwh\\Desktop\\ImageCompare\\200205_102357_FS-L01_+153_501ms_repaired162.pci_raw'

In [23]:
wkdir = Path(r"C:\Users\ucasbwh\Desktop\20200205_100115_GeomCal\PROC")
file = "Unproc_HKTM.pickle"
RTM = pd.read_pickle(wkdir / file)
RTM.head()

Time  \
0  2020-02-05\t10:01:35.440   
1  2020-02-05\t10:01:36.440   
2  2020-02-05\t10:01:37.441   
3  2020-02-05\t10:01:38.439   
4  2020-02-05\t10:01:39.441   

                                                 RAW   Source  
0  15 07 00 07 24 7F 03 CF 00 00 4D 00 07 18 04 8...  LabView  
1  15 07 00 07 24 80 03 CF 00 00 4D 00 07 17 04 8...  LabView  
2  15 07 00 07 24 81 03 CF 00 00 4D 00 07 18 04 8...  LabView  
3  15 07 00 07 24 82 03 CF 00 00 4D 00 07 18 04 8...  LabView  
4  15 07 00 07 24 83 03 CF 00 00 4D 00 07 17 04 8...  LabView

In [27]:
Bin = RTM['RAW'].apply(lambda x: bytearray.fromhex(x))
TM = pd.DataFrame()
Bin.head()

0    [21, 7, 0, 7, 36, 127, 3, 207, 0, 0, 77, 0, 7,...
1    [21, 7, 0, 7, 36, 128, 3, 207, 0, 0, 77, 0, 7,...
2    [21, 7, 0, 7, 36, 129, 3, 207, 0, 0, 77, 0, 7,...
3    [21, 7, 0, 7, 36, 130, 3, 207, 0, 0, 77, 0, 7,...
4    [21, 7, 0, 7, 36, 131, 3, 207, 0, 0, 77, 0, 7,...
Name: RAW, dtype: object

In [38]:
from PC_Fns import PandUPF

# Byte 42-43 PIU Cam Status                          #From PAN_TM_PIU_HKN_PCS and PAN_TM_PIU_HK_PCS
# PAN_TM_PIU_HKN_PCS_CE / PAN_TM_PIU_HK_PCS_CE
TM['Stat_PIU_En'] = PandUPF(Bin, 'u8', 42, 0)
# PAN_TM_PIU_HKN_PCS_PSS / PAN_TM_PIU_HK_PCS_PSS
TM['Stat_PIU_Pw'] = PandUPF(Bin, 'u8', 43, 0)
TM[40:].head(50)

Stat_PIU_En  Stat_PIU_Pw
40            3            0
41            3            0
42            3            0
43            3            0
44            3            0
45            3            0
46            3            0
47            3            0
48            3            0
49            3            3
50            3            3
51            3            3
52            3            3
53            3            3
54            3            3
55            3            3
56            3            3
57            3            3
58            3            3
59            3            3
60            3            3
61            3            3
62            3            3
63            3            3
64            3            3
65            3            3
66            3            3
67            3            3
68            3            3
69            3            3
70            3            3
71            3            3
72            3            3
73            3            3
74            3            3
75            3            3
76            3            3
77            3            3
78            3            3
79            3            3
80            3            3
81            3            3
82            3            3
83            3            3
84            3            3
85            3            3
86            3            3
87            3            3
88            3            3
89            3            3

In [61]:
# First get CamRes
CamRes = Bin.apply(lambda x: x[44:64])
CamRes[40:].head(50)

40    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
41    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
42    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
43    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
44    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
45    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
46    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
47    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
48    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
49    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
50    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
51    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
52    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
53    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
54    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
55    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
56    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
57    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [111]:
# Determine if CamRes changed
camres_chg = CamRes != CamRes.shift(1)
camres_chg[0] = False
camres_chg.head(60)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
59    False
Name: RAW, dtype: bool

In [117]:
# HRC Data is when CamRes has changed and Powered Cam is 3
HRCBin = Bin[camres_chg & (TM['Stat_PIU_Pw'] == 3)]
HRCBin.head(20)

75     [53, 3, 0, 7, 36, 202, 3, 207, 0, 0, 61, 0, 7,...
84     [53, 3, 0, 7, 36, 211, 3, 207, 0, 0, 61, 0, 7,...
87     [53, 3, 0, 7, 36, 214, 3, 207, 0, 0, 61, 0, 7,...
89     [53, 3, 0, 7, 36, 216, 3, 207, 0, 0, 61, 0, 7,...
93     [53, 3, 0, 7, 36, 220, 3, 207, 0, 0, 61, 0, 7,...
97     [53, 3, 0, 7, 36, 224, 3, 207, 0, 0, 61, 0, 7,...
101    [53, 3, 0, 7, 36, 228, 3, 207, 0, 0, 61, 0, 7,...
102    [53, 3, 0, 7, 36, 229, 3, 207, 0, 0, 61, 0, 7,...
105    [53, 3, 0, 7, 36, 232, 3, 207, 0, 0, 61, 0, 7,...
110    [53, 3, 0, 7, 36, 237, 3, 207, 0, 0, 61, 0, 7,...
114    [53, 3, 0, 7, 36, 241, 3, 207, 0, 0, 61, 0, 7,...
118    [53, 3, 0, 7, 36, 245, 3, 207, 0, 0, 61, 0, 7,...
120    [53, 3, 0, 7, 36, 247, 3, 207, 0, 0, 61, 0, 7,...
121    [53, 3, 0, 7, 36, 248, 3, 207, 0, 0, 61, 0, 7,...
122    [53, 3, 0, 7, 36, 249, 3, 207, 0, 0, 61, 0, 7,...
123    [53, 3, 0, 7, 36, 250, 3, 207, 0, 0, 61, 0, 7,...
124    [53, 3, 0, 7, 36, 251, 3, 207, 0, 0, 61, 0, 7,...
125    [53, 3, 0, 7, 36, 252, 3

In [119]:
# WAC Data is when CamRes has changed and Powered Cam is 1 or 2
WACBin = Bin[camres_chg & (TM['Stat_PIU_Pw'].between(1,2))]
WACBin.head(20)

262    [53, 3, 0, 7, 37, 133, 3, 207, 0, 0, 61, 0, 7,...
266    [53, 3, 0, 7, 37, 137, 3, 207, 0, 0, 61, 0, 7,...
278    [53, 3, 0, 7, 37, 149, 3, 207, 0, 0, 61, 0, 7,...
310    [53, 3, 0, 7, 37, 181, 3, 207, 0, 0, 61, 0, 7,...
314    [53, 3, 0, 7, 37, 185, 3, 207, 0, 0, 61, 0, 7,...
328    [53, 3, 0, 7, 37, 199, 3, 207, 0, 0, 61, 0, 7,...
385    [53, 3, 0, 7, 38, 0, 3, 207, 0, 0, 61, 0, 7, 2...
388    [53, 3, 0, 7, 38, 3, 3, 207, 0, 0, 61, 0, 7, 2...
394    [53, 3, 0, 7, 38, 9, 3, 207, 0, 0, 61, 0, 7, 2...
423    [53, 3, 0, 7, 38, 38, 3, 207, 0, 0, 61, 0, 7, ...
458    [53, 3, 0, 7, 38, 73, 3, 207, 0, 0, 61, 0, 7, ...
459    [53, 3, 0, 7, 38, 74, 3, 207, 0, 0, 61, 0, 7, ...
464    [53, 3, 0, 7, 38, 79, 3, 207, 0, 0, 61, 0, 7, ...
478    [53, 3, 0, 7, 38, 93, 3, 207, 0, 0, 61, 0, 7, ...
480    [53, 3, 0, 7, 38, 95, 3, 207, 0, 0, 61, 0, 7, ...
484    [53, 3, 0, 7, 38, 99, 3, 207, 0, 0, 61, 0, 7, ...
517    [53, 3, 0, 7, 38, 132, 3, 207, 0, 0, 61, 0, 7,...
520    [53, 3, 0, 7, 38, 135, 3

In [123]:
# Verify Null Bin is empty
NulBin = Bin[camres_chg & (TM['Stat_PIU_Pw']==0)]
NulBin.shape[0] == 0


True

In [135]:
# Verify that there is no overlap between WACBin and HRCBin

result = WACBin.to_frame().join(HRCBin.to_frame(), lsuffix='WAC', rsuffix='HRC', how='inner')
result.shape[0] == 0

True

In [17]:
wkdir = Path(r"C:\Users\ucasbwh\Desktop\20200205_100115_GeomCal")

In [18]:
archive_name = wkdir / "ARCHIVE"
target = wkdir / "ARCHIVE"
shutil.make_archive(archive_name, 'gztar', target)

KeyboardInterrupt: 

In [11]:
tar = tarfile.open(wkdir / "Archive3.tar.gz", "w:gz")
tar.add(wkdir / "ARCHIVE", arcname="ARCHIVE")
tar.close()

KeyboardInterrupt: 

In [15]:
tar = tarfile.open(wkdir / "Archive.tar.bz2", "w|bz2")
tar.add(wkdir / "ARCHIVE")
tar.close()

KeyboardInterrupt: 

In [10]:
tar = tarfile.open(wkdir / "Archive.tar.xz", "w|xz")
tar.add(wkdir / "PROC")
tar.close()

In [45]:
tmpfile = Path(r"C:\Users\ucasbwh\Desktop\Test\PROC\tmp_cmp.bin")
binfile = Path(r"C:\Users\ucasbwh\Desktop\Test\200204_144923_FS-L07_+203_252ms.bin)

SyntaxError: EOL while scanning string literal (<ipython-input-45-968db4db9683>, line 2)